In [1]:
import rpwf
from rpwf import database, rpwf
from pathlib import Path
from dataclasses import dataclass
import pandas
from sklearn.model_selection import (
    RepeatedStratifiedKFold,
    GridSearchCV,
    StratifiedKFold,
    cross_val_score
)

In [2]:
tmp_dir = Path("/tmp/Rtmpbw9gYv/vignette3b733bb9bbe4")
tmp_dir_posix = str(tmp_dir.as_posix())
print(tmp_dir_posix)

/tmp/Rtmpbw9gYv/vignette3b733bb9bbe4


In [3]:
@dataclass
class TestArgs:
    db_name: str
    project_root: str
    cores: int
    inner_n_cv: int
    outer_n_cv: int
    outer_n_repeats: int

In [4]:
args = TestArgs("db.SQLite", tmp_dir_posix, 1, 1, 1, 1)
wflow_id = 1

In [5]:
db_obj = database.Base(args.project_root, args.db_name)
wflow_df = db_obj.all_wflow()

db is at /tmp/Rtmpbw9gYv/vignette3b733bb9bbe4/rpwfDb/db.SQLite
Connecting to sqlite:////tmp/Rtmpbw9gYv/vignette3b733bb9bbe4/rpwfDb/db.SQLite


In [6]:
wflow_obj = rpwf.Wflow(db_obj, wflow_id)
n_cores = args.cores

# Generate the parameters
p_grid = rpwf.RGrid(db_obj, wflow_obj).get_grid()

df_obj = rpwf.TrainDf(db_obj, wflow_obj)
X, y = df_obj.get_df_X(), df_obj.get_df_y()

model_type_obj = rpwf.Model(db_obj, wflow_obj)
base_learner = rpwf.BaseLearner(wflow_obj, model_type_obj).base_learner
score = wflow_obj._get_par("costs")

Running <class 'xgboost.sklearn.XGBClassifier'>


/home/hhp/miniconda3/envs/py39/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [ ]:
from numpy import ravel

In [11]:
y.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 1000 entries, 1 to 1000
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   class   1000 non-null   float64
dtypes: float64(1)
memory usage: 15.6 KB


In [ ]:
# Nested resampling
inner_cv = StratifiedKFold(
    n_splits=args.inner_n_cv, shuffle=True, random_state=wflow_obj.random_state
)
outer_cv = RepeatedStratifiedKFold(
    n_splits=args.outer_n_cv,
    n_repeats=args.outer_n_repeats,
    random_state=wflow_obj.random_state,
)

if p_grid is None:
    print("No tune grid specified, running with default params")
    nested_score = cross_val_score(
        base_learner, X=X, y=y, cv=outer_cv, n_jobs=n_cores, scoring=score
    )

else:
    print("Performing nested-cv using provided Rgrid")
    param_tuner = GridSearchCV(
        estimator=base_learner,
        param_grid=p_grid,
        cv=inner_cv,
        n_jobs=n_cores,
        scoring=score,
    )
    nested_score = cross_val_score(param_tuner, X=X, y=y, cv=outer_cv)

# if args.export:
    # Export the results
exporter = rpwf.Export(db_obj, wflow_obj)
nested_score_df = pandas.DataFrame(nested_score, columns=[score])
exporter.export_cv(nested_score_df, "nested_cv")
exporter.export_db()

In [ ]:
%run -m rpwf.script.nested_resampling -h

In [ ]:
%run -m rpwf.script.nested_resampling .. -db base_db.SQLite -w 3

In [ ]:
%run -m rpwf.script.nested_resampling .. -db base_db.SQLite -w 3

In [ ]:
%run -m rpwf.script.cross_validation .. -db base_db.SQLite -w 1 -efj